# MNIST 손글씨 숫자 분류 딥러닝 모델

## 🎯 프로젝트 개요
이 노트북에서는 MNIST 데이터셋을 사용하여 손글씨 숫자(0-9)를 분류하는 다층 퍼셉트론(MLP) 딥러닝 모델을 구현합니다.

## 📚 학습 목표
- TensorFlow/Keras를 사용한 딥러닝 모델 구현
- MNIST 데이터셋 로드 및 전처리
- Sequential 모델을 이용한 신경망 구조 설계
- 모델 학습 및 성능 평가

## 📁 데이터셋 정보
- **데이터셋**: MNIST 손글씨 숫자 데이터셋
- **훈련 데이터**: 60,000개의 28x28 픽셀 이미지
- **테스트 데이터**: 10,000개의 28x28 픽셀 이미지
- **클래스**: 0~9 (10개 클래스)

## 1. 라이브러리 임포트 및 시드 설정

딥러닝 모델 구현에 필요한 라이브러리를 임포트하고, 재현 가능한 결과를 위해 랜덤 시드를 설정합니다.

In [1]:
# 필요한 라이브러리 임포트
from tensorflow.keras.datasets import mnist  # MNIST 데이터셋
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

# 재현 가능한 결과를 위한 랜덤 시드 설정
tf.random.set_seed(1234)

print("TensorFlow 버전:", tf.__version__)
print("라이브러리 임포트 완료!")

TensorFlow 버전: 2.19.0
라이브러리 임포트 완료!


## 2. MNIST 데이터셋 로드 및 탐색

MNIST 손글씨 숫자 데이터셋을 로드하고 데이터의 구조와 형태를 확인합니다.

In [2]:
# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 타입 확인
print("데이터 타입:")
print(f"train_images: {type(train_images)}")
print(f"train_labels: {type(train_labels)}")

# 데이터 형태(shape) 확인
print("\n데이터 형태:")
print(f"훈련 이미지: {train_images.shape}")  # (60000, 28, 28)
print(f"훈련 라벨: {train_labels.shape}")    # (60000,)
print(f"테스트 이미지: {test_images.shape}") # (10000, 28, 28)
print(f"테스트 라벨: {test_labels.shape}")   # (10000,)

print(f"\n총 데이터 개수: {len(train_images) + len(test_images):,}개")
print("✅ 데이터 로드 완료!")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
데이터 타입:
train_images: <class 'numpy.ndarray'>
train_labels: <class 'numpy.ndarray'>

데이터 형태:
훈련 이미지: (60000, 28, 28)
훈련 라벨: (60000,)
테스트 이미지: (10000, 28, 28)
테스트 라벨: (10000,)

총 데이터 개수: 70,000개
✅ 데이터 로드 완료!


## 3. 신경망 모델 구조 설계

Sequential 모델을 사용하여 다층 퍼셉트론(MLP) 구조를 설계합니다.

### 모델 구조:
- **입력층**: 784개 뉴런 (28x28 픽셀을 평면화)
- **은닉층 1**: 256개 뉴런 + ReLU 활성화 함수
- **은닉층 2**: 256개 뉴런 + ReLU 활성화 함수  
- **은닉층 3**: 128개 뉴런 + ReLU 활성화 함수
- **은닉층 4**: 64개 뉴런 + ReLU 활성화 함수
- **출력층**: 10개 뉴런 + Softmax 활성화 함수 (다중분류)

In [3]:
# Sequential 모델을 사용하여 신경망 구조 설계
model = keras.Sequential([
    # 은닉층 1: 256개 뉴런, ReLU 활성화 함수
    # Dense 레이어의 매개변수: (출력 뉴런 수, 활성화 함수)
    layers.Dense(256, activation='relu'),
    
    # 은닉층 2: 256개 뉴런, ReLU 활성화 함수
    layers.Dense(256, activation='relu'),
    
    # 은닉층 3: 128개 뉴런, ReLU 활성화 함수
    layers.Dense(128, activation='relu'),
    
    # 은닉층 4: 64개 뉴런, ReLU 활성화 함수
    layers.Dense(64, activation='relu'),
    
    # 출력층: 10개 뉴런 (0~9 숫자), Softmax 활성화 함수
    # 다중분류에서는 출력층에 반드시 softmax 함수 사용
    # softmax: 각 클래스에 대한 확률을 출력 (합이 1이 됨)
    layers.Dense(10, activation='softmax')
])

print("✅ 신경망 모델 구조 설계 완료!")
print("\n참고:")
print("- 회귀 문제: 출력층 → layers.Dense(1)")
print("- 이진분류: 출력층 → layers.Dense(1, activation='sigmoid')")
print("- 다중분류: 출력층 → layers.Dense(클래스수, activation='softmax')")

✅ 신경망 모델 구조 설계 완료!

참고:
- 회귀 문제: 출력층 → layers.Dense(1)
- 이진분류: 출력층 → layers.Dense(1, activation='sigmoid')
- 다중분류: 출력층 → layers.Dense(클래스수, activation='softmax')


## 4. 모델 컴파일 설정

모델을 학습할 수 있도록 옵티마이저, 손실함수, 평가지표를 설정합니다.

### 설정 요소:
- **옵티마이저**: RMSprop (가중치 업데이트 방법)
- **손실함수**: sparse_categorical_crossentropy (다중분류용)
- **평가지표**: accuracy (정확도)

In [4]:
# 모델 컴파일 설정
model.compile(
    optimizer='rmsprop',                      # 옵티마이저: RMSprop
    loss='sparse_categorical_crossentropy',   # 손실함수: 다중분류용
    metrics=['accuracy']                      # 평가지표: 정확도
)

print("✅ 모델 컴파일 완료!")
print("\n📝 손실함수 비교:")
print("- sparse_categorical_crossentropy: 라벨이 정수 형태 (0, 1, 2, ...)")
print("- categorical_crossentropy: 라벨이 원-핫 인코딩 형태 ([1,0,0], [0,1,0], ...)")
print("  → 현재는 라벨이 정수 형태이므로 sparse_categorical_crossentropy 사용")

✅ 모델 컴파일 완료!

📝 손실함수 비교:
- sparse_categorical_crossentropy: 라벨이 정수 형태 (0, 1, 2, ...)
- categorical_crossentropy: 라벨이 원-핫 인코딩 형태 ([1,0,0], [0,1,0], ...)
  → 현재는 라벨이 정수 형태이므로 sparse_categorical_crossentropy 사용


## 5. 데이터 전처리 (차원 변환 및 정규화)

딥러닝 모델 학습을 위해 데이터를 전처리합니다.

### 전처리 과정:
1. **차원 변환**: 3차원(28×28×1) → 2차원(784×1)로 평면화
2. **정규화**: 픽셀 값을 0~255 범위에서 0~1 범위로 스케일링

In [5]:
# 전처리 전 데이터 형태 확인
print("전처리 전 데이터 형태:")
print(f"훈련 이미지: {train_images.shape}")
print(f"테스트 이미지: {test_images.shape}")
print(f"픽셀 값 범위: {train_images.min()} ~ {train_images.max()}")

# 1. 차원 변환: 3차원 → 2차원 (28×28 → 784)
train_images = train_images.reshape(train_images.shape[0], 28 * 28)
test_images = test_images.reshape(test_images.shape[0], 28 * 28)

# 2. 정규화: 픽셀 값을 0~1 범위로 스케일링 (딥러닝에서는 필수!)
train_images = train_images.astype(float) / 255
test_images = test_images.astype(float) / 255

# 전처리 후 데이터 형태 확인
print("\n전처리 후 데이터 형태:")
print(f"훈련 이미지: {train_images.shape}")
print(f"테스트 이미지: {test_images.shape}")
print(f"정규화된 픽셀 값 범위: {train_images.min():.1f} ~ {train_images.max():.1f}")

print("\n✅ 데이터 전처리 완료!")
print("📝 정규화가 중요한 이유: 딥러닝에서는 입력값의 스케일이 학습에 큰 영향을 미침")

전처리 전 데이터 형태:
훈련 이미지: (60000, 28, 28)
테스트 이미지: (10000, 28, 28)
픽셀 값 범위: 0 ~ 255

전처리 후 데이터 형태:
훈련 이미지: (60000, 784)
테스트 이미지: (10000, 784)
정규화된 픽셀 값 범위: 0.0 ~ 1.0

✅ 데이터 전처리 완료!
📝 정규화가 중요한 이유: 딥러닝에서는 입력값의 스케일이 학습에 큰 영향을 미침


## 6. 모델 학습 실행

fit() 함수를 사용하여 모델을 학습시킵니다.

### 학습 매개변수:
- **epochs**: 100 (전체 데이터셋을 100번 반복 학습)
- **batch_size**: 128 (한 번에 128개 샘플씩 메모리에 로드)

In [6]:
# 모델 학습 실행
print("🚀 모델 학습을 시작합니다...")
print("⏰ 학습 시간: 약 5-10분 소요 예상")

hist = model.fit(
    train_images,    # X: 독립변수, 입력 데이터
    train_labels,    # y: 종속변수, 타겟 라벨
    epochs=100,      # 학습 회수 (전체 데이터셋을 100번 반복)
    batch_size=128   # 배치 크기 (메모리 효율성을 위해 128개씩 처리)
)

print("\n✅ 모델 학습 완료!")
print("\n📝 배치 크기(batch_size) 설명:")
print("- 너무 크면: 메모리 부족 발생 가능")
print("- 너무 작으면: 학습 속도가 느려짐")
print("- 적절한 크기: 32, 64, 128, 256 등의 2의 거듭제곱 값 사용")

🚀 모델 학습을 시작합니다...
⏰ 학습 시간: 약 5-10분 소요 예상
Epoch 1/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8366 - loss: 0.5156
Epoch 2/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9640 - loss: 0.1171
Epoch 3/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9773 - loss: 0.0732
Epoch 4/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9840 - loss: 0.0516
Epoch 5/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9884 - loss: 0.0378
Epoch 6/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9911 - loss: 0.0292
Epoch 7/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9928 - loss: 0.0223
Epoch 8/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9936 - loss: 0.0202
Epoch 9/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9951 - loss: 0.0173
Epoch 10/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9951 - loss: 0.0156
Epoch 11/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9956 - loss: 0.0139


## 7. 모델 성능 평가

evaluate() 함수를 사용하여 훈련셋과 테스트셋에서의 모델 성능을 평가합니다.

In [7]:
# 훈련셋 성능 평가
train_loss, train_acc = model.evaluate(train_images, train_labels, verbose=0)
print("📊 훈련셋 성능:")
print(f"   손실(Loss): {train_loss:.4f}")
print(f"   정확도(Accuracy): {train_acc:.4f} ({train_acc*100:.2f}%)")

# 테스트셋 성능 평가
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print("\n📊 테스트셋 성능:")
print(f"   손실(Loss): {test_loss:.4f}")
print(f"   정확도(Accuracy): {test_acc:.4f} ({test_acc*100:.2f}%)")

# 성능 차이 분석
accuracy_diff = train_acc - test_acc
print(f"\n📈 성능 분석:")
print(f"   정확도 차이: {accuracy_diff:.4f} ({accuracy_diff*100:.2f}%p)")

if accuracy_diff > 0.05:  # 5% 이상 차이
    print("   ⚠️  과적합(Overfitting) 가능성이 있습니다.")
else:
    print("   ✅ 적절한 일반화 성능을 보입니다.")

print("\n🎉 MNIST 손글씨 분류 모델 구현 완료!")

📊 훈련셋 성능:
   손실(Loss): 0.0000
   정확도(Accuracy): 1.0000 (100.00%)

📊 테스트셋 성능:
   손실(Loss): 0.2185
   정확도(Accuracy): 0.9836 (98.36%)

📈 성능 분석:
   정확도 차이: 0.0164 (1.64%p)
   ✅ 적절한 일반화 성능을 보입니다.

🎉 MNIST 손글씨 분류 모델 구현 완료!


## 📚 학습 요약 및 추가 개선 방안

### ✅ 구현한 내용:
1. **MNIST 데이터셋 로드 및 탐색**
2. **다층 퍼셉트론(MLP) 모델 설계**
3. **데이터 전처리 (차원변환, 정규화)**
4. **모델 학습 및 성능 평가**

### 🔧 모델 개선 방안:
1. **CNN(Convolutional Neural Network) 사용**: 이미지 특성을 더 잘 학습
2. **Dropout 추가**: 과적합 방지
3. **Batch Normalization**: 학습 안정성 향상
4. **Learning Rate Scheduling**: 학습률 조정
5. **Data Augmentation**: 데이터 증강으로 일반화 성능 향상

### 🎯 다음 단계:
- **Scikit-learn의 Iris 데이터셋으로 딥러닝 구현**
- **CNN을 사용한 이미지 분류**
- **다른 데이터셋(CIFAR-10 등) 실습**

### 📝 딥러닝 모델 설계 패턴:
- **회귀**: `Dense(1)` - 연속값 예측
- **이진분류**: `Dense(1, activation='sigmoid')` - 0 또는 1
- **다중분류**: `Dense(클래스수, activation='softmax')` - 여러 클래스 중 하나